In [1]:
!gdown 10NrO_YQ4vqeJ3uyp5A0z2GJjnJWenc4s
!unzip  flower_photos.zip

Downloading...
From: https://drive.google.com/uc?id=10NrO_YQ4vqeJ3uyp5A0z2GJjnJWenc4s
To: /content/flower_photos.zip
100% 230M/230M [00:01<00:00, 169MB/s]
Archive:  flower_photos.zip
replace flower_photos/train/sunflowers/15118243470_7e0a7f159c_n.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.autograd import Variable
import os
import numpy as np
from PIL import Image
from sklearn import preprocessing

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

# Transforms to be applied to Train-Test-Validation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(means, stds)])


In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, img_path):
        self.img_path = img_path

        self.data = []
        self.label = []
        for i in os.listdir(self.img_path):
            if os.path.isdir(self.img_path + "/" + i):
                for j in os.listdir(self.img_path + "/" + i):
                    self.data.append(self.img_path + "/" + i + "/" + j)
                    self.label.append(str(i))
        label_encoder = preprocessing.LabelEncoder()
        self.label = label_encoder.fit_transform(self.label)
        mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
        print(mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(self.data[idx])
        img = train_transforms(img)
        label = self.label[idx]
        label_tensor = torch.as_tensor(label, dtype=torch.long)
        return {'im': img, 'labels': label_tensor}


In [6]:
# load data
train_dataset = CustomImageDataset('/content/flower_photos/train')
test_dataset = CustomImageDataset('/content/flower_photos/test')

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}
{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [7]:
train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=32,
                                           shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                                          batch_size=4,
                                          shuffle=False)


In [8]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Model().to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(200):
    running_loss=0
    running_acc=0
   
    for i, data in enumerate(train_loader):
        inputs, labels = data['im'].to(device), data['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        result = torch.argmax(outputs,dim=1)
        running_loss+=loss.item()
        running_acc+=torch.mean((result==labels).type(torch.float))
        
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
      train_loss=running_loss/len(train_loader)
      train_acc=running_acc/len(train_loader)
      print('epoch {}, loss {}'.format(epoch, train_loss))


epoch 0, loss 2.2333874272870586
epoch 10, loss 1.2663533075435742
epoch 20, loss 1.1670920687752802
epoch 30, loss 1.0714523668761726
epoch 40, loss 0.9826546354336781
epoch 50, loss 0.9577107075098399
epoch 60, loss 0.9400485796971364
epoch 70, loss 0.9074533318614101
epoch 80, loss 0.8910749721097516
epoch 90, loss 0.8418254739529377
epoch 100, loss 0.8706301668205777
epoch 110, loss 0.8441367122504089
epoch 120, loss 0.8249531450035336
epoch 130, loss 0.8267104663290419
epoch 140, loss 0.8099122952233564
epoch 150, loss 0.8196809554959202
epoch 160, loss 0.7863693629299198
epoch 170, loss 0.7858730758632626
epoch 180, loss 0.7829949745723793
epoch 190, loss 0.7615860058917655


In [11]:
model.eval()  
correct = 0
total = 0
for i, data in enumerate(test_loader):
    inputs, labels = data['im'].to(device), data['labels'].to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the test images: %d %%' % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 75 %
